# Turbine Operation Models

Separate from the turbine models, which define the physical characterstics of the turbines, FLORIS
allows users to specify how the turbine behaves in terms of producing power and thurst. We refer to 
different models for turbine behavior as "operation models". A key feature of operation models is
the ability for users to specify control setpoints at which the operation model will be evaluated. 
For instance, some operation models allow users to specify `yaw_angles`, which alter the power 
being produced by the turbine along with it's thrust force on flow.

Operation models are specified by the `operation_model` key on the turbine yaml file, or by using
the `set_operation_model()` method on `FlorisModel`. Each operation model available in FLORIS is
described and demonstrated below. The simplest operation model is the `"simple"` operation model,
which takes no control setpoints and simply evaluates the power and thrust coefficient curves for 
the turbine at the current wind condition. The default operation model is the `"cosine-loss"`
operation model, which models the loss in power of a turbine under yaw misalignment using a cosine
term with an exponent.

We first provide a quick demonstration of how to switch between different operation models. Then, 
each operation model available in FLORIS is described, along with its relevant control setpoints.
We also describe the different parameters that must be specified in the turbine 
`"power_thrust_table"` dictionary in order to use that operation model.

## Selecting the operation model

There are two options for selecting the operation model:
1. Manually changing the `"operation_model"` field of the turbine input yaml 
(see [Turbine Input File Reference](input_reference_turbine))

2. Using `set_operation_model()` on an instantiated `FlorisModel` object.

The following code demonstrates the use of the second option.

In [ ]:
from floris import FlorisModel
from floris import layout_visualization as layoutviz

fmodel = FlorisModel("../examples/inputs/gch.yaml")

# Look at layout
ax = layoutviz.plot_turbine_rotors(fmodel)
layoutviz.plot_turbine_labels(fmodel, ax=ax)
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")

# Set simple operation model
fmodel.set_operation_model("simple")

# Evalaute the model and extract the power output
fmodel.run()
print("simple operation model powers [kW]: ", fmodel.get_turbine_powers() / 1000)

# Set the yaw angles (which the "simple" operation model does not use
# and change the operation model to "cosine-loss"
fmodel.set(yaw_angles=[[20., 0., 0.]])
fmodel.set_operation_model("cosine-loss")
ax = layoutviz.plot_turbine_rotors(fmodel)
layoutviz.plot_turbine_labels(fmodel, ax=ax)
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")

# Evaluate again
fmodel.run()
powers_cosine_loss = fmodel.get_turbine_powers()
print("cosine-loss operation model powers [kW]: ", fmodel.get_turbine_powers() / 1000)


## Operation model library

### Simple model
User-level name: `"simple"`

Underlying class: `SimpleTurbine`

Required data on `power_thrust_table`:
- `ref_air_density` (scalar)
- `ref_tilt` (scalar)
- `wind_speed` (list)
- `power` (list)
- `thrust_coefficient` (list)

The `"simple"` operation model describes the "normal" function of a wind turbine, as described by
its power curve and thrust coefficient. It does not respond to any control setpoints, and is most 
often used as a baseline or for users wanting to evaluate wind farms in nominal operation.

### Cosine loss model
User-level name: `"cosine-loss"`

Underlying class: `CosineLossTurbine`

Required data on `power_thrust_table`:
- `ref_air_density` (scalar)
- `ref_tilt` (scalar)
- `wind_speed` (list)
- `power` (list)
- `thrust_coefficient` (list)
- `cosine_loss_exponent_yaw` (scalar)
- `cosine_loss_exponent_tilt` (scalar)

The `"cosine-loss"` operation model describes the decrease in power and thrust produced by a 
wind turbine as it yaws (or tilts) away from the incoming wind. The thrust is reduced by a factor of 
$\cos \gamma$, where $\gamma$ is the yaw misalignment angle, while the power is reduced by a factor 
of $(\cos\gamma)^{p_P}$, where $p_P$ is the cosine loss exponent, specified by `cosine_loss_exponent_yaw`
(or `cosine_loss_exponent_tilt` for tilt angles). The power and thrust produced by the turbine
thus vary as a function of the turbine's yaw angle, set using the `yaw_angles` argument to 
`FlorisModel.set()`.

In [ ]:
from floris import TimeSeries
import numpy as np
import matplotlib.pyplot as plt

# Set up the FlorisModel
fmodel.set_operation_model("cosine-loss")
fmodel.set(layout_x=[0.0], layout_y=[0.0])
fmodel.set(
    wind_data=TimeSeries(
        wind_speeds=np.ones(100) * 8.0,
        wind_directions=np.ones(100) * 270.0,
        turbulence_intensities=0.06
    )
)
fmodel.reset_operation()

# Sweep the yaw angles
yaw_angles = np.linspace(-25, 25, 100)
fmodel.set(yaw_angles=yaw_angles.reshape(-1,1))
fmodel.run()

powers = fmodel.get_turbine_powers()/1000

fig, ax = plt.subplots()
ax.plot(yaw_angles, powers)
ax.grid()
ax.set_xlabel("Yaw angle [deg]")
ax.set_ylabel("Power [kW]")

### Simple derating model
User-level name: `"simple-derating"`

Underlying class: `SimpleDeratingTurbine`

Required data on `power_thrust_table`:
- `ref_air_density` (scalar)
- `ref_tilt` (scalar)
- `wind_speed` (list)
- `power` (list)
- `thrust_coefficient` (list)

The `"simple-derating"` operation model enables users to derate turbines by setting a new power 
rating. It does not require any extra parameters on the `power_thrust_table`, but adescribes the 
decrease in power and thrust produced by providing the `power_setpoints` argument to
`FlorisModel.set()`. The default power rating for the turbine can be acheived by setting the
appropriate entries of `power_setpoints` to `None`.

In [ ]:
# Set up the FlorisModel
fmodel.set_operation_model("simple-derating")
fmodel.reset_operation()
wind_speeds = np.linspace(0, 30, 100)
fmodel.set(
    wind_data=TimeSeries(
        wind_speeds=wind_speeds,
        wind_directions=np.ones(100) * 270.0,
        turbulence_intensities=0.06
    )
)

fig, ax = plt.subplots()
for power_setpoint in [5.0, 4.0, 3.0, 2.0]:
    fmodel.set(power_setpoints=np.array([[power_setpoint*1e6]]*100))
    fmodel.run()
    powers = fmodel.get_turbine_powers()/1000
    ax.plot(wind_speeds, powers[:,0], label=f"Power setpoint (MW): {power_setpoint}")

ax.grid()
ax.legend()
ax.set_xlabel("Wind speed [m/s]")
ax.set_ylabel("Power [kW]")

### Mixed operation model
User-level name: `"mixed"`

Underlying class: `MixedOperationTurbine`

Required data on `power_thrust_table`:
- `ref_air_density` (scalar)
- `ref_tilt` (scalar)
- `wind_speed` (list)
- `power` (list)
- `thrust_coefficient` (list)
- `cosine_loss_exponent_yaw` (scalar)
- `cosine_loss_exponent_tilt` (scalar)

The `"mixed"` operation model allows users to specify _either_ `yaw_angles` (evaluated using the 
`"cosine-loss"` operation model) _or_ `power_setpoints` (evaluated using the `"simple-derating"`
operation model). That is, for each turbine, and at each `findex`, a non-zero yaw angle or a 
non-`None` power setpoint may be specified. However, specifying both a non-zero yaw angle and a 
finite power setpoint for the same turbine and at the same `findex` will produce an error.

In [ ]:
fmodel.set_operation_model("mixed")
fmodel.set(layout_x=[0.0, 0.0], layout_y=[0.0, 500.0])
fmodel.reset_operation()
fmodel.set(
    wind_data=TimeSeries(
        wind_speeds=np.array([10.0]),
        wind_directions=np.array([270.0]),
        turbulence_intensities=0.06
    )
)
fmodel.set(
    yaw_angles=np.array([[20.0, 0.0]]),
    power_setpoints=np.array([[None, 2e6]])
)
fmodel.run()
print("Powers [kW]: ", fmodel.get_turbine_powers()/1000)

### AWC model

User-level name: `"awc"`

Underlying class: `AWCTurbine`

Required data on `power_thrust_table`:
- `ref_air_density` (scalar)
- `ref_tilt` (scalar)
- `wind_speed` (list)
- `power` (list)
- `thrust_coefficient` (list)
- `helix_a` (scalar)
- `helix_power_b` (scalar)
- `helix_power_c` (scalar)
- `helix_thrust_b` (scalar)
- `helix_thrust_c` (scalar)

The `"awc"` operation model allows for users to define _active wake control_ strategies. These strategies 
use pitch control to actively enhance wake mixing and subsequently decrease wake velocity deficits. As a 
result, downstream turbines can increase their power production, with limited power loss for the controlled 
upstream turbine. The `AWCTurbine` class models this power loss at the turbine applying AWC. For each 
turbine, the user can define an AWC strategy to implement through the `awc_modes` array. Note that currently, 
only `"baseline"`, i.e., no AWC, and `"helix"`, i.e., the 
[counterclockwise helix method](https://doi.org/10.1002/we.2513) have been implemented. 

The user then defines the exact AWC implementation through setting the variable `awc_amplitudes` for 
each turbine. This variable defines the mean-to-peak amplitude of the sinusoidal AWC pitch excitation,
i.e., for a turbine that under `awc_modes = "baseline"` has a constant pitch angle of 0 degrees, setting 
`awc_amplitude = 2` results in a pitch signal varying from -2 to 2 degrees over the desired Strouhal
frequency. This Strouhal frequency is not used as an input here, since it has minimal influence on turbine 
power production. Note that setting `awc_amplitudes = 0` effectively disables AWC and is therefore the same 
as running a turbine at `awc_modes = "baseline"`.

Each example turbine input file `floris/turbine_library/*.yaml` has its own `helix_*` parameter data. These 
parameters are determined by fitting data from `OpenFAST` simulations in region II to the following equation:

$$
    P_\text{AWC} = P_\text{baseline} \cdot (1 - (b + c \cdot P_\text{baseline} ) \cdot A_\text{AWC}^a)
$$

where $a$ is `"helix_a"`, $b$ is `"helix_power_b"`, $c$ is `"helix_power_c"`, and $A_\text{AWC}$ is `awc_amplitudes`. 
The thrust coefficient follows the same equation, but with the respective thrust parameters. When AWC is 
turned on while $P_\text{baseline} > P_\text{rated}$, a warning is given as the model is not yet tuned for region III.

The figure below shows the fit between the turbine power and thrust in OpenFAST helix AWC simulations (x) 
and FLORIS simulations (--) at different region II wind speeds for the NREL 5MW reference turbine.

<!--<img src="powerthrust_helix.png" width="600px">-->
![](./powerthrust_helix.png)

In [ ]:
fmodel = FlorisModel("../examples/inputs/emgauss_helix.yaml")
fmodel.set_operation_model("awc")
fmodel.set(layout_x=[0.0, 0.0], layout_y=[0.0, 500.0])
fmodel.reset_operation()
fmodel.set(
    wind_speeds=np.array([8.0]),
    wind_directions=np.array([270.0]),
    turbulence_intensities=np.array([0.06])
)
fmodel.set(
    awc_modes=np.array([["helix", "baseline"]]),
    awc_amplitudes=np.array([[2.5, 0]])
)
fmodel.run()
print("Powers [kW]: ", fmodel.get_turbine_powers()/1000)

### Peak shaving model

User-level name: `"peak-shaving"`

Underlying class: `PeakShavingTurbine`

Required data on `power_thrust_table`:
- `ref_air_density` (scalar)
- `ref_tilt` (scalar)
- `wind_speed` (list)
- `power` (list)
- `thrust_coefficient` (list)
- `peak_shaving_fraction` (scalar)
- `peak_shaving_TI_threshold` (scalar)

The `"peak-shaving"` operation model allows users to implement peak shaving, where the thrust
of the wind turbine is reduced from the nominal curve near rated to reduce unwanted structural
loading. Peak shaving here is implemented here by reducing the thrust by a fixed fraction from
the peak thrust on the nominal thrust curve, as specified by `peak_shaving_fraction`.This only
affects wind speeds near the peak in the thrust
curve (usually near rated wind speed), as thrust values away from the peak will be below the
fraction regardless. Further, peak shaving is only applied if the turbulence intensity experienced
by the turbine meets the `peak_shaving_TI_threshold`. To apply peak shaving in all wind conditions,
`peak_shaving_TI_threshold` may be set to zero.

When the turbine is peak shaving to reduce thrust, the power output is updated accordingly. Letting
$C_{T}$ represent the thrust coefficient when peak shaving (at given wind speed), and $C_{T}'$
represent the thrust coefficient that the turbine would be operating at under nominal control, then
the power $P$ due to peak shaving (compared to the power $P'$ available under nominal control) is 
computed (based on actuator disk theory) as

$$ P = \frac{C_T (1 - a)}{C_T' (1 - a')} P'$$

where $a$ (respectively, $a'$) is the axial induction factor corresponding to $C_T$
(respectively, $C_T'$), computed using the usual relationship from actuator disk theory,
i.e. the lesser solution to $C_T=4a(1-a)$.


In [ ]:
# Set up the FlorisModel
fmodel = FlorisModel("../examples/inputs/gch.yaml")
fmodel.set(
    layout_x=[0.0], layout_y=[0.0],
    wind_data=TimeSeries(
        wind_speeds=wind_speeds,
        wind_directions=np.ones(100) * 270.0,
        turbulence_intensities=0.2 # Higher than threshold value of 0.1
    )
)
fmodel.reset_operation()
fmodel.set_operation_model("simple")
fmodel.run()
powers_base = fmodel.get_turbine_powers()/1000
thrust_coefficients_base = fmodel.get_turbine_thrust_coefficients()
fmodel.set_operation_model("peak-shaving")
fmodel.run()
powers_peak_shaving = fmodel.get_turbine_powers()/1000
thrust_coefficients_peak_shaving = fmodel.get_turbine_thrust_coefficients()

fig, ax = plt.subplots(2,1,sharex=True)
ax[0].plot(wind_speeds, thrust_coefficients_base, label="Without peak shaving", color="black")
ax[0].plot(wind_speeds, thrust_coefficients_peak_shaving, label="With peak shaving", color="C0")
ax[1].plot(wind_speeds, powers_base, label="Without peak shaving", color="black")
ax[1].plot(wind_speeds, powers_peak_shaving, label="With peak shaving", color="C0")

ax[1].grid()
ax[0].grid()
ax[0].legend()
ax[0].set_ylabel("Thrust coefficient [-]")
ax[1].set_xlabel("Wind speed [m/s]")
ax[1].set_ylabel("Power [kW]")